In [1]:
# ЯЧЕЙКА - Встроенный setup
def setup_environment():
    """Полная настройка окружения прямо в ноутбуке"""
    import subprocess
    import sys

    packages = [
        'pandas', 'numpy', 'matplotlib', 'seaborn',
        'scikit-learn', 'sentence-transformers', 'transformers',
        'torch', 'faiss-cpu', 'annoy'
    ]

    for package in packages:
        print(f"Устанавливаю {package}...")
        !pip install -q {package}

    print("🎉 Все пакеты установлены!")

setup_environment()

Устанавливаю pandas...
Устанавливаю numpy...
Устанавливаю matplotlib...
Устанавливаю seaborn...
Устанавливаю scikit-learn...
Устанавливаю sentence-transformers...
Устанавливаю transformers...
Устанавливаю torch...
Устанавливаю faiss-cpu...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 45.5 MB/s eta 0:00:00
Устанавливаю annoy...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
🎉 Все пакеты установлены!


In [2]:
# ЯЧЕЙКА 2 - Тестовые данные книг
import pandas as pd
import numpy as np

# Создаем небольшой тестовый датасет книг
books_data = {
    'title': [
        'Гарри Поттер и философский камень',
        '1984',
        'Мастер и Маргарита',
        'Властелин Колец: Братство Кольца',
        'Убить пересмешника',
        'Три товарища',
        'Анна Каренина',
        'Преступление и наказание',
        'Маленький принц',
        'Тёмные начала'
    ],
    'author': [
        'Дж. К. Роулинг',
        'Джордж Оруэлл',
        'Михаил Булгаков',
        'Дж. Р. Р. Толкин',
        'Харпер Ли',
        'Эрих Мария Ремарк',
        'Лев Толстой',
        'Фёдор Достоевский',
        'Антуан де Сент-Экзюпери',
        'Филип Пулман'
    ],
    'year': [1997, 1949, 1967, 1954, 1960, 1936, 1877, 1866, 1943, 1995],
    'genre': [
        'фэнтези', 'антиутопия', 'мистика', 'фэнтези', 'драма',
        'роман', 'роман', 'психологический роман', 'притча', 'фэнтези'
    ],
    'mood': [
        'приключенческое', 'мрачное', 'сатирическое', 'эпическое', 'трогательное',
        'грустное', 'трагическое', 'психологическое', 'философское', 'приключенческое'
    ],
    'description': [
        'Мальчик-сирота узнает, что он волшебник и отправляется в школу магии Хогвартс',
        'О жизни под постоянным контролем Большого Брата в тоталитарном обществе',
        'Визит дьявола в Москву 1930-х годов и его встречи с литературной элитой',
        'Хоббит Фродо должен уничтожить опасное магическое кольцо в огне Роковой горы',
        'История о расовой несправедливости и детской невинности в американском юге',
        'История дружбы трех молодых людей в Германии между двумя мировыми войнами',
        'Трагическая история замужней аристократки и её романа с богатым офицером',
        'История бывшего студента, совершившего убийство и пытающегося оправдать свой поступок',
        'Философская сказка о пилоте, встретившем в пустыне мальчика с другой планеты',
        'Приключения девочки Лиры в параллельном мире, где души людей существуют вне их тел'
    ]
}

# Создаем DataFrame
df_books = pd.DataFrame(books_data)
print("📚 Наш книжный каталог:")
print(f"Всего книг: {len(df_books)}")
print("Первые 3 книги:")
display(df_books.head(3))

# Сохраняем в CSV для будущего использования
df_books.to_csv('books_dataset.csv', index=False, encoding='utf-8')
print("✅ Датасет сохранен как 'books_dataset.csv'")

📚 Наш книжный каталог:
Всего книг: 10
Первые 3 книги:


,title,author,year,genre,mood,description
0,Гарри Поттер и философский камень,Дж. К. Роулинг,1997,фэнтези,приключенческое,"Мальчик-сирота узнает, что он волшебник и отпр..."
1,1984,Джордж Оруэлл,1949,антиутопия,мрачное,О жизни под постоянным контролем Большого Брат...
2,Мастер и Маргарита,Михаил Булгаков,1967,мистика,сатирическое,Визит дьявола в Москву 1930-х годов и его встр...


✅ Датасет сохранен как 'books_dataset.csv'


In [3]:
# ЯЧЕЙКА 3 - Книжный AI-агент
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class BookRecommendationAgent:
    def __init__(self):
        # Загружаем модель для преобразования текста в векторы
        print("🔄 Загружаю модель для обработки текста...")
        self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        self.books_df = None
        self.book_embeddings = None
        print("✅ Модель загружена!")

    def load_books(self, books_df):
        """Загружаем книги в агента"""
        self.books_df = books_df
        print(f"📚 Загружено {len(books_df)} книг")

    def train(self):
        """Создаем эмбеддинги для всех книг"""
        if self.books_df is None:
            print("❌ Сначала загрузите книги!")
            return

        print("🔄 Создаю векторные представления книг...")

        # Создаем текстовые описания для векторного представления
        book_texts = []
        for _, row in self.books_df.iterrows():
            text = f"{row['title']} {row['genre']} {row['mood']} {row['description']}"
            book_texts.append(text)

        # Преобразуем в векторы
        self.book_embeddings = self.model.encode(book_texts)
        print(f"✅ Модель обучена! Создано {len(self.book_embeddings)} векторных представлений.")

    def recommend_books(self, genre, mood, n_books=5):
        """Рекомендуем книги по жанру и настроению"""
        if self.book_embeddings is None:
            print("❌ Сначала обучите модель!")
            return []

        # Создаем запрос пользователя
        user_query = f"{genre} {mood}"
        print(f"🔍 Ищу: {user_query}")

        # Преобразуем запрос в вектор
        query_embedding = self.model.encode([user_query])

        # Вычисляем схожесть с каждой книгой
        similarities = cosine_similarity(query_embedding, self.book_embeddings)[0]

        # Получаем индексы самых похожих книг
        top_indices = np.argsort(similarities)[::-1][:n_books]

        # Формируем рекомендации
        recommendations = []
        for idx in top_indices:
            book = self.books_df.iloc[idx]
            recommendations.append({
                'title': book['title'],
                'author': book['author'],
                'year': book['year'],
                'genre': book['genre'],
                'mood': book['mood'],
                'description': book['description'],
                'similarity_score': similarities[idx]
            })

        return recommendations

# Создаем и обучаем нашего агента
print("🤖 Создаю книжного AI-агента...")
book_agent = BookRecommendationAgent()
book_agent.load_books(df_books)
book_agent.train()
print("✅ Агент готов к работе!")

🤖 Создаю книжного AI-агента...
🔄 Загружаю модель для обработки текста...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Модель загружена!
📚 Загружено 10 книг
🔄 Создаю векторные представления книг...
✅ Модель обучена! Создано 10 векторных представлений.
✅ Агент готов к работе!


In [4]:
# ЯЧЕЙКА 4 - Тестируем рекомендации

def print_recommendations(recommendations):
    """Красиво выводим рекомендации"""
    print("📚 РЕКОМЕНДАЦИИ:")
    print("=" * 50)
    for i, book in enumerate(recommendations, 1):
        print(f"{i}. {book['title']}")
        print(f" 👤 Автор: {book['author']}")
        print(f" 📅 Год: {book['year']}")
        print(f" 🎭 Жанр: {book['genre']}")
        print(f" 💫 Настроение: {book['mood']}")
        print(f" 📖 Описание: {book['description'][:100]}...")
        print(f" 🔍 Схожесть: {book['similarity_score']:.3f}")
        print()

# Тест 1: Фэнтези с приключениями
print("🎯 Запрос: фэнтези приключенческое")
recs = book_agent.recommend_books("фэнтези", "приключенческое")
print_recommendations(recs)

# Тест 2: Мрачные книги
print("🎯 Запрос: роман трагическое")
recs = book_agent.recommend_books("роман", "трагическое")
print_recommendations(recs)

🎯 Запрос: фэнтези приключенческое
🔍 Ищу: фэнтези приключенческое
📚 РЕКОМЕНДАЦИИ:
1. Тёмные начала
 👤 Автор: Филип Пулман
 📅 Год: 1995
 🎭 Жанр: фэнтези
 💫 Настроение: приключенческое
 📖 Описание: Приключения девочки Лиры в параллельном мире, где души людей существуют вне их тел...
 🔍 Схожесть: 0.769

2. Гарри Поттер и философский камень
 👤 Автор: Дж. К. Роулинг
 📅 Год: 1997
 🎭 Жанр: фэнтези
 💫 Настроение: приключенческое
 📖 Описание: Мальчик-сирота узнает, что он волшебник и отправляется в школу магии Хогвартс...
 🔍 Схожесть: 0.757

3. Властелин Колец: Братство Кольца
 👤 Автор: Дж. Р. Р. Толкин
 📅 Год: 1954
 🎭 Жанр: фэнтези
 💫 Настроение: эпическое
 📖 Описание: Хоббит Фродо должен уничтожить опасное магическое кольцо в огне Роковой горы...
 🔍 Схожесть: 0.675

4. Маленький принц
 👤 Автор: Антуан де Сент-Экзюпери
 📅 Год: 1943
 🎭 Жанр: притча
 💫 Настроение: философское
 📖 Описание: Философская сказка о пилоте, встретившем в пустыне мальчика с другой планеты...
 🔍 Схожесть: 0.616

5. Убить 

In [ ]:
# ЯЧЕЙКА 5 - Интерактивный режим

def interactive_mode():
    """Режим взаимодействия с пользователем"""
    print("🤖 Добро пожаловать в книжного AI-агента!")
    print("Я помогу подобрать книги по вашему настроению и жанру")
    print("=" * 50)

    while True:
        print("\nВведите параметры для поиска:")
        genre = input("🎭 Жанр (например: фэнтези, роман, детектив): ").strip()
        mood = input("💫 Настроение (например: приключенческое, мрачное, романтическое): ").strip()

        if not genre or not mood:
            print("❌ Пожалуйста, введите и жанр, и настроение")
            continue

        print(f"🔍 Ищу книги: {genre}, {mood}...")
        recommendations = book_agent.recommend_books(genre, mood)

        if recommendations:
            print_recommendations(recommendations)
        else:
            print("😔 К сожалению, не нашлось подходящих книг")

        # Спросим, хочет ли пользователь продолжить
        continue_search = input("Продолжить поиск? (да/нет): ").strip().lower()
        if continue_search not in ['да', 'д', 'yes', 'y']:
            print("📚 Приятного чтения! До свидания!")
            break

# Запускаем интерактивный режим
interactive_mode()

🤖 Добро пожаловать в книжного AI-агента!
Я помогу подобрать книги по вашему настроению и жанру

Введите параметры для поиска:
🎭 Жанр (например: фэнтези, роман, детектив): роман
💫 Настроение (например: приключенческое, мрачное, романтическое): мрачное
🔍 Ищу книги: роман, мрачное...
🔍 Ищу: роман мрачное
📚 РЕКОМЕНДАЦИИ:
1. Анна Каренина
 👤 Автор: Лев Толстой
 📅 Год: 1877
 🎭 Жанр: роман
 💫 Настроение: трагическое
 📖 Описание: Трагическая история замужней аристократки и её романа с богатым офицером...
 🔍 Схожесть: 0.651

2. Три товарища
 👤 Автор: Эрих Мария Ремарк
 📅 Год: 1936
 🎭 Жанр: роман
 💫 Настроение: грустное
 📖 Описание: История дружбы трех молодых людей в Германии между двумя мировыми войнами...
 🔍 Схожесть: 0.617

3. Мастер и Маргарита
 👤 Автор: Михаил Булгаков
 📅 Год: 1967
 🎭 Жанр: мистика
 💫 Настроение: сатирическое
 📖 Описание: Визит дьявола в Москву 1930-х годов и его встречи с литературной элитой...
 🔍 Схожесть: 0.559

4. Убить пересмешника
 👤 Автор: Харпер Ли
 📅 Год: 1960
 🎭 